In [ ]:
%matplotlib inline
import copy

import numpy as np
import torch
import torch.nn.utils.prune as prune
from torch.nn import CrossEntropyLoss
import matplotlib.pyplot as plt
cmap = plt.get_cmap("viridis")
from tqdm import tqdm

import connectome
config = connectome.get_config()

from connectome.core.data_processing import DataProcessor
from connectome.core.graph_models import FullGraphModel
from connectome.core.utils import (
    clean_model_outputs, 
    get_image_paths, 
    get_iteration_number, 
    initialize_results_df, 
    select_random_images, 
    update_results_df, 
    update_running_loss
)
from connectome.visualization.plots import plot_results

device_type = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device(device_type)
dtype = torch.float32
batch_size = config.batch_size

In [ ]:
data_processor = DataProcessor(config)
model = FullGraphModel(data_processor, config).to(device)
criterion = CrossEntropyLoss()

In [ ]:
def correct_test_results(test_results):
    # There was a bug in how we get the classes, and the 0-1 labels can be flipped
    # This function corrects the labels if the accuracy is below 0.5
    flipped = False
    if test_results["Is correct"].sum() / len(test_results) < 0.5:
        test_results["Is correct"] = np.abs(test_results["Is correct"] - 1)
        flipped = True

    return test_results, flipped

# test
def test(model):
    testing_images = get_image_paths(
        config.TESTING_DATA_DIR, config.small_length
    )
    already_selected_testing = []
    total_correct, total, running_loss = 0, 0, 0.0
    test_results = initialize_results_df()

    iterations = get_iteration_number(len(testing_images), batch_size)
    for _ in tqdm(range(iterations)):
        batch_files, already_selected_testing = select_random_images(
            testing_images, batch_size, already_selected_testing
        )
        images, labels = data_processor.get_data_from_paths(batch_files)
        inputs, labels = data_processor.process_batch(images, labels)
        inputs = inputs.to(config.DEVICE)

        out = model(inputs)
        loss = criterion(out, labels)

        # Calculate run parameters
        outputs, predictions, labels_cpu, correct = clean_model_outputs(out, labels)
        test_results = update_results_df(
            test_results, batch_files, outputs, predictions, labels_cpu, correct
        )
        running_loss += update_running_loss(loss, inputs)
        total += batch_size
        total_correct += correct.sum()

    test_results, flipped = correct_test_results(test_results)
    final_plots = plot_results(test_results, plot_types=config.plot_types)

    print(
        f"Finished testing with loss {running_loss / total} and "
        f"accuracy {total_correct / total}."
    )
    return test_results, final_plots, total_correct / total, flipped

In [ ]:
checkpoint = torch.load(
    "models/n_all_v_R7_r_False_lr_0.001_p_4_2024-05-23 18:58.pth",
    map_location=device_type,
)
model.load_state_dict(checkpoint["model"])
torch.set_grad_enabled(False)
model.eval()
model_orig = copy.deepcopy(model)


In [ ]:
results = {}
for ix, p in enumerate(np.linspace(0, 1, 11)):
    model = copy.deepcopy(model_orig)
    prune.random_unstructured(
        model.connectome, name="edge_weight_multiplier", amount=p
    )
    test_results, final_plots, accuracy, flipped = test(model)
    results[f"prune_{ix}"] = {
        "prune_frac": p,
        "test_results": test_results,
        "final_plots": final_plots,
        "accuracy": accuracy,
        "flipped": flipped,
    }

In [ ]:
import pandas as pd
pa = pd.DataFrame({"prune_frac": [r["prune_frac"] for r in results.values()], "accuracy": [1 - r["accuracy"] for r in results.values()]})

In [ ]:
%matplotlib inline
import seaborn as sns

fig = plt.figure(figsize=(10, 6))
sns.lineplot(data=pa, x="prune_frac", y="accuracy")
plt.show()